# **Classificando o dataset MNIST usando MLPs e CNN**

### Membros

* Antônio Barros
* André Vasconcelos
* Heitor Santos
* João Vítor Valadares
* Robson Oliveira

###Introdução
Esse relatório fala sobre o desenvolvimento de uma solução para o problema de classificação de dígitos escritos manualmente do dataset MNIST, dividido em duas partes, a primeira usando Redes Neurais Multilayer Perceptron (MLP) e a segunda Redes Neurais Convolucionais (CNN). O dataset MNIST consiste em 70 mil imagens 28x28 dos dígitos de 0 a 9, sendo 60 mil samples de treinamento e 10 mil samples de teste.

##Bibliotecas
Utilizamos basicamente para o treinamento, modelagem da nossa Rede Neural, o conjunto de bibliotecas do Tensorflow e Keras. E o Numpy para representar dados e funções.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD, Adam

##Processando e Carregando Dado

Carregamos o MNIST dataset já vetorizado em formato numpy e separado em vetores de treinamento: *x_train, y_train* que contém o vetor de imagens e valores esperado respectivamente; e em vetores de teste: *x_test, y_test*, que contém o vetor de imagens e valores esperado respectivamente. 

Normalizamos as imagens ao formato desejado (uint8 para float32), para ficar no intervalo de [0,1] dividimos o vetor de imagens por 255 (uint8 vai do intervalo [0,255]). E convertamos também para uma matriz de classe binária, os vetores de classe.

---



In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print("{0} samples para treinamento".format(len(x_train)))
print("{0} samples para teste".format(len(x_test)))

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

11501568/11490434 [==============================] - 0s 0us/step
60000 samples para treinamento
10000 samples para teste


## Função de Avaliação
Para avaliar o desempenho da nossa Rede Neural, criamos duas funções de avaliação, *accuracy_class* responsável por avaliar a acurácia sobre cada classe de resposta, que são os digitos entre 0 a 9. E a *accuracy_total* responsável que retorna as seguintes métricas: **acurácia**, **precision** e **recall**.

Definição das métricas:

* *accuracy*:  Indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente.
* *precision*: Entre todas as classificações de classe Positivo que o modelo fez, quantas estão corretas.
* *recall*: Dentre todas as situações de classe Positivo como valor esperado, quantas estão corretas.

In [ ]:
def accuracy_class(model):
    predicts = model.predict(x_test)
    correctArray = [0,0,0,0,0,0,0,0,0,0]
    total = [0,0,0,0,0,0,0,0,0,0]
    for correct, predict in zip(y_test.argmax(axis=1), predicts.argmax(axis=1)):
        if correct == predict:
            correctArray[correct] += 1
        total[correct] += 1

    print("Acurácia por Classe")
    for x in range(10):
      percent = (correctArray[x]/total[x])*100
      print("Classe {0}: {1:.6f}%".format(x,percent))
tf.keras
def accuracy_total(model):
    _, acc, prec, rec = model.evaluate(x_test, y_test,verbose=0)

    print("Acurácia Total: {0:.6f}".format(acc))
    print("Precision:      {0:.6f}".format(prec))
    print("Recall:         {0:.6f}".format(rec))

## Primeira versão: MLP

Vamos defirnir uma MLP com as seguintes restrições:

*  Uma camada de entrada com um neuron para cada um dos pixels da imagem;
*  Uma Hidden Layer com 30 neurônios
* E uma Output Layer com 10 neurônios, cada um indicando um digito.

E essa Primeira versão da MLP vai ser modificando de acordo com as próximas experimentações

In [ ]:
model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(30),
    layers.Dense(10, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 30)                23550     
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
Total params: 23,860
Trainable params: 23,860
Non-trainable params: 0
_________________________________________________________________


## Parâmetros para treinamento
Definimos alguns para parâmetros para o treinamento da Rede Neural, a escolha foi feita se baseando nos parâmetros dados inicial pelo [link](http://neuralnetworksanddeeplearning.com/chap1.html), com o intuito de atingir maior perfomance

In [ ]:
model.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.1)

Epoch 1/30
5400/5400 [==============================] - 15s 2ms/step - loss: 0.4645 - accuracy: 0.8712 - precision: 0.9317 - recall: 0.8053 - val_loss: 0.2790 - val_accuracy: 0.9227 - val_precision: 0.9432 - val_recall: 0.9023
Epoch 2/30
5400/5400 [==============================] - 8s 2ms/step - loss: 0.3207 - accuracy: 0.9085 - precision: 0.9312 - recall: 0.8893 - val_loss: 0.2525 - val_accuracy: 0.9298 - val_precision: 0.9500 - val_recall: 0.9153
Epoch 3/30
5400/5400 [==============================] - 8s 1ms/step - loss: 0.3024 - accuracy: 0.9146 - precision: 0.9341 - recall: 0.8974 - val_loss: 0.2454 - val_accuracy: 0.9285 - val_precision: 0.9468 - val_recall: 0.9162
Epoch 4/30
5400/5400 [==============================] - 9s 2ms/step - loss: 0.2932 - accuracy: 0.9178 - precision: 0.9348 - recall: 0.9026 - val_loss: 0.2420 - val_accuracy: 0.9307 - val_precision: 0.9486 - val_recall: 0.9195
Epoch 5/30
5400/5400 [==============================] - 8s 2ms/step - loss: 0.2868 - accuracy: 

In [ ]:
accuracy_class(model)

Acurácia por Classe
Classe 0: 97.755102%
Classe 1: 98.149780%
Classe 2: 88.565891%
Classe 3: 90.990099%
Classe 4: 92.362525%
Classe 5: 87.780269%
Classe 6: 95.302714%
Classe 7: 91.731518%
Classe 8: 89.733060%
Classe 9: 89.494549%


In [ ]:
accuracy_total(model)

Acurácia Total: 0.922800
Precision:      0.937346
Recall:         0.914100


Com o termino do treinamento e da avaliação podemos perceber essa primeira versão da nossa Rede já atinge uma grande parte dos dados do dataset.

##Experimentações

A partir do nosso modelo básico de MLP, vamos ir modificando ele para tentar atingir maiores níveis de acerto, modificando: tamanho da rede, taxa
de aprendizagem, função de ativação, algoritmo de aprendizagem,
drop-out

## Segunda versão: MLP

Vamos verificar se aumentando o batch_size de 10 para 128, vamos ter um aumento no desempenho.

In [ ]:
model2 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(30),
    layers.Dense(10, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 30)                23550     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                310       
Total params: 23,860
Trainable params: 23,860
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.fit(x_train, y_train, batch_size=128, epochs=30, validation_split=0.1)

Epoch 1/30
422/422 [==============================] - 1s 3ms/step - loss: 0.2896 - accuracy: 0.9185 - precision_2: 0.9375 - recall_2: 0.9017 - val_loss: 0.2446 - val_accuracy: 0.9312 - val_precision_2: 0.9475 - val_recall_2: 0.9177
Epoch 2/30
422/422 [==============================] - 1s 2ms/step - loss: 0.2887 - accuracy: 0.9189 - precision_2: 0.9376 - recall_2: 0.9025 - val_loss: 0.2445 - val_accuracy: 0.9313 - val_precision_2: 0.9473 - val_recall_2: 0.9168
Epoch 3/30
422/422 [==============================] - 1s 3ms/step - loss: 0.2875 - accuracy: 0.9190 - precision_2: 0.9382 - recall_2: 0.9026 - val_loss: 0.2435 - val_accuracy: 0.9305 - val_precision_2: 0.9477 - val_recall_2: 0.9175
Epoch 4/30
422/422 [==============================] - 1s 3ms/step - loss: 0.2867 - accuracy: 0.9192 - precision_2: 0.9381 - recall_2: 0.9034 - val_loss: 0.2432 - val_accuracy: 0.9313 - val_precision_2: 0.9493 - val_recall_2: 0.9177
Epoch 5/30
422/422 [==============================] - 1s 3ms/step - loss

In [ ]:
accuracy_class(model2)

Acurácia por Classe
Classe 0: 97.959184%
Classe 1: 97.444934%
Classe 2: 89.341085%
Classe 3: 91.287129%
Classe 4: 92.464358%
Classe 5: 86.098655%
Classe 6: 95.198330%
Classe 7: 93.093385%
Classe 8: 89.117043%
Classe 9: 89.593657%


In [ ]:
accuracy_total(model2)

Acurácia Total: 0.922700
Precision:      0.939121
Recall:         0.908600


Percebemos que apesar que o aumento do batch agilizou o treinamento da nossa Rede, ela foi ligeraimente pior que a nossa primeira versão, na acurácia total temos uma diferença de 0.0001 ou 0.01%


## Terceira versão: MLP
No terceiro experimento nós substítuimos a função de ativação da output layer para a função sigmoide.

In [ ]:
model3 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(30),
    layers.Dense(10, activation="sigmoid")
])

model3.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model3.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_21 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 30)                23550     
_________________________________________________________________
dense_61 (Dense)             (None, 10)                310       
Total params: 23,860
Trainable params: 23,860
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.1) # mudou a função de ativação e mantivemos os outros parametros

Epoch 1/30
422/422 [==============================] - 2s 4ms/step - loss: 1.1645 - accuracy: 0.6975 - precision_19: 0.2058 - recall_19: 0.9349 - val_loss: 0.6209 - val_accuracy: 0.8643 - val_precision_19: 0.2259 - val_recall_19: 0.9872
Epoch 2/30
422/422 [==============================] - 1s 3ms/step - loss: 0.5913 - accuracy: 0.8499 - precision_19: 0.2195 - recall_19: 0.9846 - val_loss: 0.4372 - val_accuracy: 0.8953 - val_precision_19: 0.2219 - val_recall_19: 0.9900
Epoch 3/30
422/422 [==============================] - 1s 3ms/step - loss: 0.4791 - accuracy: 0.8721 - precision_19: 0.2171 - recall_19: 0.9884 - val_loss: 0.3708 - val_accuracy: 0.9052 - val_precision_19: 0.2185 - val_recall_19: 0.9915
Epoch 4/30
422/422 [==============================] - 1s 3ms/step - loss: 0.4287 - accuracy: 0.8832 - precision_19: 0.2153 - recall_19: 0.9902 - val_loss: 0.3373 - val_accuracy: 0.9120 - val_precision_19: 0.2160 - val_recall_19: 0.9928
Epoch 5/30
422/422 [==============================] - 1s

In [ ]:
accuracy_class(model3)

Acurácia por Classe
Classe 0: 97.755102%
Classe 1: 97.533040%
Classe 2: 89.147287%
Classe 3: 89.702970%
Classe 4: 93.177189%
Classe 5: 86.210762%
Classe 6: 95.198330%
Classe 7: 91.634241%
Classe 8: 88.706366%
Classe 9: 89.296333%


In [ ]:
accuracy_total(model3)

Acurácia Total: 0.919400
Precision:      0.204384
Recall:         0.994900


Percebemos que os valores diminuíram após usar essa função, isso acontece porque a função sigmoide é melhor para classificação binária, já que ela considera os valores de forma independente, por isso a precisão ficou muito baixa.

## Quarta versão: MLP

Nesse experimento, nós adicionamos mais uma hidden layer, uma de quinze neurônios.

In [ ]:
model4 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(30),
    layers.Dense(15),
    layers.Dense(10, activation="softmax") 
])

model4.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 30)                23550     
_________________________________________________________________
dense_1 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
Total params: 24,175
Trainable params: 24,175
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model4.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.1) # mudou a função de ativação e mantivemos os outros parametros

Epoch 1/30
5400/5400 [==============================] - 10s 2ms/step - loss: 0.4585 - accuracy: 0.8703 - precision: 0.9247 - recall: 0.8157 - val_loss: 0.2765 - val_accuracy: 0.9172 - val_precision: 0.9375 - val_recall: 0.9043
Epoch 2/30
5400/5400 [==============================] - 9s 2ms/step - loss: 0.3202 - accuracy: 0.9084 - precision: 0.9296 - recall: 0.8917 - val_loss: 0.2516 - val_accuracy: 0.9252 - val_precision: 0.9435 - val_recall: 0.9123
Epoch 3/30
5400/5400 [==============================] - 9s 2ms/step - loss: 0.3038 - accuracy: 0.9136 - precision: 0.9320 - recall: 0.8986 - val_loss: 0.2512 - val_accuracy: 0.9282 - val_precision: 0.9446 - val_recall: 0.9178
Epoch 4/30
5400/5400 [==============================] - 10s 2ms/step - loss: 0.2957 - accuracy: 0.9164 - precision: 0.9331 - recall: 0.9030 - val_loss: 0.2563 - val_accuracy: 0.9273 - val_precision: 0.9411 - val_recall: 0.9140
Epoch 5/30
5400/5400 [==============================] - 10s 2ms/step - loss: 0.2892 - accuracy

In [ ]:
accuracy_class(model4)

Acurácia por Classe
Classe 0: 97.142857%
Classe 1: 98.237885%
Classe 2: 90.019380%
Classe 3: 90.495050%
Classe 4: 92.973523%
Classe 5: 86.547085%
Classe 6: 96.033403%
Classe 7: 92.023346%
Classe 8: 86.652977%
Classe 9: 91.774034%


In [ ]:
accuracy_total(model4) 

Acurácia Total: 0.923100
Precision:      0.936744
Recall:         0.913700


A camada a mais fez diminuir a acurácia em 0.09% e a precision também diminiu em 0.30%, porém o recall aumentou em 0.11%.

## Quinta versão: MLP
Nesse experimento, nós decidimos aumentar significativamente o número de neurônios em cada hidden layer.

In [ ]:
model5 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(500),
    layers.Dense(250),
    layers.Dense(10, activation="softmax")
])

model5.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model5.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_4 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2510      
Total params: 520,260
Trainable params: 520,260
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model5.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.1) 

Epoch 1/30
5400/5400 [==============================] - 28s 5ms/step - loss: 0.3891 - accuracy: 0.8879 - precision_1: 0.9249 - recall_1: 0.8514 - val_loss: 0.2655 - val_accuracy: 0.9238 - val_precision_1: 0.9419 - val_recall_1: 0.9103
Epoch 2/30
5400/5400 [==============================] - 28s 5ms/step - loss: 0.3123 - accuracy: 0.9118 - precision_1: 0.9303 - recall_1: 0.8956 - val_loss: 0.2701 - val_accuracy: 0.9238 - val_precision_1: 0.9397 - val_recall_1: 0.9117
Epoch 3/30
5400/5400 [==============================] - 26s 5ms/step - loss: 0.2995 - accuracy: 0.9159 - precision_1: 0.9326 - recall_1: 0.9014 - val_loss: 0.2464 - val_accuracy: 0.9300 - val_precision_1: 0.9460 - val_recall_1: 0.9200
Epoch 4/30
5400/5400 [==============================] - 24s 4ms/step - loss: 0.2928 - accuracy: 0.9183 - precision_1: 0.9335 - recall_1: 0.9042 - val_loss: 0.2512 - val_accuracy: 0.9282 - val_precision_1: 0.9423 - val_recall_1: 0.9152
Epoch 5/30
5400/5400 [==============================] - 25s 

In [ ]:
accuracy_class(model5)

Acurácia por Classe
Classe 0: 96.530612%
Classe 1: 97.533040%
Classe 2: 89.728682%
Classe 3: 92.475248%
Classe 4: 92.260692%
Classe 5: 86.995516%
Classe 6: 93.423800%
Classe 7: 93.093385%
Classe 8: 89.630390%
Classe 9: 91.080278%


In [ ]:
accuracy_total(model5) 

Acurácia Total: 0.923900
Precision:      0.937109
Recall:         0.913400


Percebemos que a acurácia total e a precisão diminuíram, porém o recall aumentou.

## Sexta versão: MLP
Nós percebemos que o processo estava muito lento e não estava necessariamente ganhando muito desempenho nos resultados, então nós decidimos aumentar novamente o batch size para 128 e aumentamos o learning rate e diminuímos o número de épocas.

In [ ]:
model6 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(500),
    layers.Dense(250),
    layers.Dense(10, activation="softmax")
])

model6.compile(loss="categorical_crossentropy", optimizer=SGD(0.2), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]) # + learning-rate

model6.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_7 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2510      
Total params: 520,260
Trainable params: 520,260
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model6.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 6s 12ms/step - loss: 0.4036 - accuracy: 0.8826 - precision_2: 0.9158 - recall_2: 0.8536 - val_loss: 0.2731 - val_accuracy: 0.9242 - val_precision_2: 0.9406 - val_recall_2: 0.9075
Epoch 2/10
422/422 [==============================] - 5s 12ms/step - loss: 0.3295 - accuracy: 0.9053 - precision_2: 0.9240 - recall_2: 0.8897 - val_loss: 0.2665 - val_accuracy: 0.9218 - val_precision_2: 0.9383 - val_recall_2: 0.9093
Epoch 3/10
422/422 [==============================] - 5s 12ms/step - loss: 0.3129 - accuracy: 0.9110 - precision_2: 0.9287 - recall_2: 0.8965 - val_loss: 0.2426 - val_accuracy: 0.9307 - val_precision_2: 0.9482 - val_recall_2: 0.9207
Epoch 4/10
422/422 [==============================] - 5s 12ms/step - loss: 0.3049 - accuracy: 0.9136 - precision_2: 0.9299 - recall_2: 0.8994 - val_loss: 0.2769 - val_accuracy: 0.9147 - val_precision_2: 0.9320 - val_recall_2: 0.9025
Epoch 5/10
422/422 [==============================] - 5s 12ms/step -

In [ ]:
accuracy_class(model6)

Acurácia por Classe
Classe 0: 96.326531%
Classe 1: 98.325991%
Classe 2: 93.604651%
Classe 3: 88.514851%
Classe 4: 88.187373%
Classe 5: 84.417040%
Classe 6: 94.676409%
Classe 7: 90.369650%
Classe 8: 88.193018%
Classe 9: 93.855302%


In [ ]:
accuracy_total(model6)

Acurácia Total: 0.918100
Precision:      0.935022
Recall:         0.908000


Esse experimento não deu muito certo, e tanto a acurácia total, quanto a precisão e recall diminuíram.

## Sétima versão: MLP
Nesse experimento, nós utilizamos o dropout, que é uma técnica de regularização para reduzir overfit e otimizar nossa rede, ele ignora aleatoriamente neurônios da camada indicada.

In [ ]:
model7 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dropout(0.1),
    layers.Dense(500),
    layers.Dropout(0.1),
    layers.Dense(250),
    layers.Dense(10, activation="softmax")
])

model7.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model7.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout (Dropout)            (None, 784)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 500)               392500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2510      
Total params: 520,260
Trainable params: 520,260
Non-trainable params: 0
________________________________________________

In [ ]:
model7.fit(x_train, y_train, batch_size=128, epochs=30, validation_split=0.1) 

Epoch 1/30
422/422 [==============================] - 7s 15ms/step - loss: 0.7622 - accuracy: 0.7934 - precision_3: 0.9374 - recall_3: 0.6055 - val_loss: 0.3830 - val_accuracy: 0.9045 - val_precision_3: 0.9563 - val_recall_3: 0.8463
Epoch 2/30
422/422 [==============================] - 6s 14ms/step - loss: 0.4438 - accuracy: 0.8734 - precision_3: 0.9238 - recall_3: 0.8208 - val_loss: 0.3139 - val_accuracy: 0.9150 - val_precision_3: 0.9503 - val_recall_3: 0.8797
Epoch 3/30
422/422 [==============================] - 6s 14ms/step - loss: 0.4005 - accuracy: 0.8836 - precision_3: 0.9219 - recall_3: 0.8464 - val_loss: 0.2894 - val_accuracy: 0.9188 - val_precision_3: 0.9478 - val_recall_3: 0.8927
Epoch 4/30
422/422 [==============================] - 6s 14ms/step - loss: 0.3752 - accuracy: 0.8922 - precision_3: 0.9242 - recall_3: 0.8623 - val_loss: 0.2773 - val_accuracy: 0.9222 - val_precision_3: 0.9482 - val_recall_3: 0.9007
Epoch 5/30
422/422 [==============================] - 6s 14ms/step -

In [ ]:
accuracy_class(model7)

Acurácia por Classe
Classe 0: 97.857143%
Classe 1: 97.797357%
Classe 2: 89.922481%
Classe 3: 90.792079%
Classe 4: 92.362525%
Classe 5: 85.986547%
Classe 6: 95.302714%
Classe 7: 91.926070%
Classe 8: 90.759754%
Classe 9: 90.782953%


In [ ]:
accuracy_total(model7)

Acurácia Total: 0.924600
Precision:      0.938866
Recall:         0.910700


Esse experimento aumentou a acurácia em 0,06%, mas diminuiu precisão e recall 

## Oitava versão: MLP

Nesse experimento, nós mudamos a função de ativação de umas das hidden layers para relu, e além disso mudamamos o algoritmo de aprendizagem para Adam, que é uma extensão do gradient descent.

In [ ]:
model8 = keras.Sequential([
    keras.Input(shape=(28,28,1)),
    layers.Flatten(),
    layers.Dropout(0.1),
    layers.Dense(500, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(250),
    layers.Dense(10, activation="softmax")
])

model8.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model8.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 500)               392500    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2510      
Total params: 520,260
Trainable params: 520,260
Non-trainable params: 0
________________________________________________

In [ ]:
model8.fit(x_train, y_train, batch_size=128, epochs=30, validation_split=0.1)

Epoch 1/30
422/422 [==============================] - 7s 17ms/step - loss: 0.0154 - accuracy: 0.9949 - precision_4: 0.9951 - recall_4: 0.9948 - val_loss: 0.0957 - val_accuracy: 0.9850 - val_precision_4: 0.9855 - val_recall_4: 0.9850
Epoch 2/30
422/422 [==============================] - 7s 16ms/step - loss: 0.0153 - accuracy: 0.9951 - precision_4: 0.9954 - recall_4: 0.9950 - val_loss: 0.1132 - val_accuracy: 0.9823 - val_precision_4: 0.9836 - val_recall_4: 0.9820
Epoch 3/30
422/422 [==============================] - 6s 15ms/step - loss: 0.0138 - accuracy: 0.9955 - precision_4: 0.9957 - recall_4: 0.9954 - val_loss: 0.0974 - val_accuracy: 0.9832 - val_precision_4: 0.9836 - val_recall_4: 0.9823
Epoch 4/30
422/422 [==============================] - 6s 15ms/step - loss: 0.0147 - accuracy: 0.9956 - precision_4: 0.9957 - recall_4: 0.9954 - val_loss: 0.0994 - val_accuracy: 0.9817 - val_precision_4: 0.9821 - val_recall_4: 0.9808
Epoch 5/30
422/422 [==============================] - 7s 16ms/step -

In [ ]:
accuracy_class(model8)

Acurácia por Classe
Classe 0: 99.285714%
Classe 1: 98.590308%
Classe 2: 98.062016%
Classe 3: 97.623762%
Classe 4: 97.657841%
Classe 5: 98.094170%
Classe 6: 98.225470%
Classe 7: 97.957198%
Classe 8: 98.254620%
Classe 9: 98.017839%


In [ ]:
accuracy_total(model8)

Acurácia Total: 0.981800
Precision:      0.982388
Recall:         0.981700


Com essas alterações, nós tivemos resultados bem satisfatórios, com a acurácia total aumentando em 6%, a precisão aumentando em 5% e o recall aumentando em 7%.

## Nona versão: MLP

Aqui nós mudamos a função de ativação da última hidden layler para sigmoid.

In [ ]:
model9 = keras.Sequential([
    keras.Input(shape=(28,28,1)),
    layers.Flatten(),
    layers.Dropout(0.1),
    layers.Dense(500, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(250, activation="sigmoid"),
    layers.Dense(10, activation="softmax")
])

model9.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model9.summary()

In [ ]:
model9.fit(x_train, y_train, batch_size=128, epochs=30, validation_split=0.1) 

Epoch 1/30
422/422 [==============================] - 8s 16ms/step - loss: 0.3456 - accuracy: 0.9022 - precision_5: 0.9448 - recall_5: 0.8539 - val_loss: 0.1227 - val_accuracy: 0.9635 - val_precision_5: 0.9706 - val_recall_5: 0.9573
Epoch 2/30
422/422 [==============================] - 6s 15ms/step - loss: 0.1323 - accuracy: 0.9606 - precision_5: 0.9690 - recall_5: 0.9538 - val_loss: 0.0828 - val_accuracy: 0.9757 - val_precision_5: 0.9800 - val_recall_5: 0.9732
Epoch 3/30
422/422 [==============================] - 6s 15ms/step - loss: 0.0903 - accuracy: 0.9717 - precision_5: 0.9768 - recall_5: 0.9676 - val_loss: 0.0726 - val_accuracy: 0.9785 - val_precision_5: 0.9820 - val_recall_5: 0.9755
Epoch 4/30
422/422 [==============================] - 6s 15ms/step - loss: 0.0691 - accuracy: 0.9779 - precision_5: 0.9815 - recall_5: 0.9756 - val_loss: 0.0642 - val_accuracy: 0.9805 - val_precision_5: 0.9838 - val_recall_5: 0.9793
Epoch 5/30
422/422 [==============================] - 6s 15ms/step -

In [ ]:
accuracy_class(model9)

Acurácia por Classe
Classe 0: 99.183673%
Classe 1: 99.559471%
Classe 2: 98.158915%
Classe 3: 97.920792%
Classe 4: 98.472505%
Classe 5: 97.645740%
Classe 6: 98.329854%
Classe 7: 98.540856%
Classe 8: 98.459959%
Classe 9: 98.612488%


In [ ]:
accuracy_total(model9)

Acurácia Total: 0.985100
Precision:      0.985787
Recall:         0.984900


Como resultado, aumentamos acurácia, precisão e recall em relação ao último experimento.

# Primeira versão: CNN

Agora, ao invés de utilizar redes MLP iniciaremos nossos testes com redes CNN que possuem uma vantagem em relação às redes MLP: considerar a estrutura da matriz de entrada.

Nessa primeira versão iniciaremos com uma CNN simples, de apenas uma camada convolucional e uma camada de pooling.

In [ ]:
model10 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

model10.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model10.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model10.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 17s 41ms/step - loss: 0.0128 - accuracy: 0.9967 - precision_1: 0.9971 - recall_1: 0.9963 - val_loss: 0.0587 - val_accuracy: 0.9853 - val_precision_1: 0.9865 - val_recall_1: 0.9853
Epoch 2/10
422/422 [==============================] - 17s 41ms/step - loss: 0.0121 - accuracy: 0.9971 - precision_1: 0.9974 - recall_1: 0.9967 - val_loss: 0.0637 - val_accuracy: 0.9853 - val_precision_1: 0.9860 - val_recall_1: 0.9845
Epoch 3/10
422/422 [==============================] - 17s 41ms/step - loss: 0.0107 - accuracy: 0.9976 - precision_1: 0.9979 - recall_1: 0.9972 - val_loss: 0.0591 - val_accuracy: 0.9852 - val_precision_1: 0.9855 - val_recall_1: 0.9845
Epoch 4/10
422/422 [==============================] - 17s 41ms/step - loss: 0.0096 - accuracy: 0.9980 - precision_1: 0.9983 - recall_1: 0.9978 - val_loss: 0.0639 - val_accuracy: 0.9857 - val_precision_1: 0.9863 - val_recall_1: 0.9853
Epoch 5/10
422/422 [==============================] - 17s 41ms/s

In [ ]:
accuracy_class(model10)

Acurácia por Classe
Classe 0: 98.877551%
Classe 1: 99.471366%
Classe 2: 97.383721%
Classe 3: 99.108911%
Classe 4: 98.778004%
Classe 5: 98.654709%
Classe 6: 98.225470%
Classe 7: 98.054475%
Classe 8: 97.741273%
Classe 9: 96.233895%


In [ ]:
accuracy_total(model10)

Acurácia Total: 0.982600
Precision:      0.982988
Recall:         0.982300


Tivemos bons resultados de acurácia, precision e recall, no entanto todos esses valores foram menores do que nossa rede MLP final testada.


## Segunda versão: CNN
Na segunda versão da CNN nós adicionamos uma camada a mais antes da úlltima, com uma função de ativação sigmóide, e com o batch size 10.

In [ ]:
model11 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(250, activation="sigmoid"),
        layers.Dense(10, activation="softmax")
    ]
)

model11.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model11.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               1352250   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2510      
Total params: 1,355,080
Trainable params: 1,355,080
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model11.fit(x_train, y_train, batch_size=10, epochs=10, validation_split=0.1)

Epoch 1/10
5400/5400 [==============================] - 69s 13ms/step - loss: 0.0326 - accuracy: 0.9891 - precision_1: 0.9900 - recall_1: 0.9882 - val_loss: 0.0468 - val_accuracy: 0.9880 - val_precision_1: 0.9895 - val_recall_1: 0.9877
Epoch 2/10
5400/5400 [==============================] - 68s 13ms/step - loss: 0.0144 - accuracy: 0.9953 - precision_1: 0.9956 - recall_1: 0.9949 - val_loss: 0.0560 - val_accuracy: 0.9867 - val_precision_1: 0.9878 - val_recall_1: 0.9865
Epoch 3/10
5400/5400 [==============================] - 68s 13ms/step - loss: 0.0101 - accuracy: 0.9969 - precision_1: 0.9972 - recall_1: 0.9967 - val_loss: 0.0508 - val_accuracy: 0.9875 - val_precision_1: 0.9882 - val_recall_1: 0.9870
Epoch 4/10
5400/5400 [==============================] - 68s 13ms/step - loss: 0.0071 - accuracy: 0.9979 - precision_1: 0.9979 - recall_1: 0.9978 - val_loss: 0.0538 - val_accuracy: 0.9875 - val_precision_1: 0.9878 - val_recall_1: 0.9872
Epoch 5/10
5400/5400 [==============================] - 

In [ ]:
accuracy_class(model11)

Acurácia por Classe
Classe 0: 99.489796%
Classe 1: 99.735683%
Classe 2: 98.934109%
Classe 3: 99.207921%
Classe 4: 98.981670%
Classe 5: 98.094170%
Classe 6: 98.643006%
Classe 7: 98.443580%
Classe 8: 97.638604%
Classe 9: 98.711596%


In [ ]:
accuracy_total(model11)

Acurácia Total: 0.988100
Precision:      0.988594
Recall:         0.988100


Percebemos então que obtemos resultados melhores que o melhor modelo anterior em todas as categorias

## Terceira versão: CNN

No próximo modelo nós testamos a função relu invés da função sigmóide, já que a função relu é a mais utilizada. Além disso, aumentamos o número de épocas para 30 e setamos o batch size para 64

In [ ]:
model12 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(250, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

model12.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model12.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               1352250   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2510      
Total params: 1,355,080
Trainable params: 1,355,080
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model12.fit(x_train, y_train, batch_size=64, epochs=30, validation_split=0.1)

Epoch 1/30
844/844 [==============================] - 30s 35ms/step - loss: 0.1701 - accuracy: 0.9513 - precision_2: 0.9675 - recall_2: 0.9339 - val_loss: 0.0616 - val_accuracy: 0.9847 - val_precision_2: 0.9858 - val_recall_2: 0.9818
Epoch 2/30
844/844 [==============================] - 30s 35ms/step - loss: 0.0557 - accuracy: 0.9825 - precision_2: 0.9850 - recall_2: 0.9804 - val_loss: 0.0493 - val_accuracy: 0.9873 - val_precision_2: 0.9890 - val_recall_2: 0.9862
Epoch 3/30
844/844 [==============================] - 30s 35ms/step - loss: 0.0349 - accuracy: 0.9893 - precision_2: 0.9906 - recall_2: 0.9883 - val_loss: 0.0432 - val_accuracy: 0.9873 - val_precision_2: 0.9893 - val_recall_2: 0.9865
Epoch 4/30
844/844 [==============================] - 29s 35ms/step - loss: 0.0238 - accuracy: 0.9924 - precision_2: 0.9931 - recall_2: 0.9920 - val_loss: 0.0460 - val_accuracy: 0.9872 - val_precision_2: 0.9880 - val_recall_2: 0.9867
Epoch 5/30
844/844 [==============================] - 29s 35ms/s

In [ ]:
accuracy_class(model12)

Acurácia por Classe
Classe 0: 99.591837%
Classe 1: 99.471366%
Classe 2: 99.031008%
Classe 3: 99.207921%
Classe 4: 99.185336%
Classe 5: 98.654709%
Classe 6: 98.851775%
Classe 7: 98.832685%
Classe 8: 98.870637%
Classe 9: 98.513380%


In [ ]:
accuracy_total(model12)

Acurácia Total: 0.990300
Precision:      0.990398
Recall:         0.990200


Percebemos nesse teste que tivemos os melhores resultados até aqui novamente.

## Quarta versão: CNN
Nessa versão nós mudamos o filters da primeira camada para 64 e o strides para (2,2), número de pixels que serão shiftados sobre nossa matrix de entrada.

In [ ]:
model13 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(250, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

model13.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model13.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 13, 13, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 250)               576250    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                2510      
Total params: 579,400
Trainable params: 579,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model13.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 13s 28ms/step - loss: 0.2794 - accuracy: 0.9195 - precision_4: 0.9549 - recall_4: 0.8821 - val_loss: 0.0958 - val_accuracy: 0.9737 - val_precision_4: 0.9780 - val_recall_4: 0.9687
Epoch 2/10
422/422 [==============================] - 12s 28ms/step - loss: 0.0940 - accuracy: 0.9709 - precision_4: 0.9768 - recall_4: 0.9667 - val_loss: 0.0745 - val_accuracy: 0.9795 - val_precision_4: 0.9826 - val_recall_4: 0.9772
Epoch 3/10
422/422 [==============================] - 12s 28ms/step - loss: 0.0668 - accuracy: 0.9794 - precision_4: 0.9830 - recall_4: 0.9765 - val_loss: 0.0629 - val_accuracy: 0.9815 - val_precision_4: 0.9841 - val_recall_4: 0.9800
Epoch 4/10
422/422 [==============================] - 12s 28ms/step - loss: 0.0510 - accuracy: 0.9842 - precision_4: 0.9868 - recall_4: 0.9825 - val_loss: 0.0586 - val_accuracy: 0.9837 - val_precision_4: 0.9848 - val_recall_4: 0.9825
Epoch 5/10
422/422 [==============================] - 12s 27ms/s

In [ ]:
accuracy_class(model13)

Acurácia por Classe
Classe 0: 99.489796%
Classe 1: 99.383260%
Classe 2: 98.837209%
Classe 3: 99.009901%
Classe 4: 98.574338%
Classe 5: 99.103139%
Classe 6: 98.956159%
Classe 7: 98.151751%
Classe 8: 98.767967%
Classe 9: 98.414272%


In [ ]:
accuracy_total(model13)

Acurácia Total: 0.988700
Precision:      0.989486
Recall:         0.988200


Aqui nós obtivemos resultados não tão bons quanto os dois anteriores, mas ainda assim muito bons

## Quinta versão: CNN
Nessa versão utilizando o nosso melhor modelo até o momento, vamos adicionar mais uma camada de convulução, já que cada camada de convolução é responsável pela a diminuição de features de entradas para as camadas ocultas.

In [ ]:
model14 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(250, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

model14.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model14.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               200250    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [ ]:
model14.fit(x_train, y_train, batch_size=64, epochs=30, validation_split=0.1)

Epoch 1/30
844/844 [==============================] - 40s 46ms/step - loss: 0.1754 - accuracy: 0.9470 - precision: 0.9684 - recall: 0.9275 - val_loss: 0.0538 - val_accuracy: 0.9862 - val_precision: 0.9881 - val_recall: 0.9833
Epoch 2/30
844/844 [==============================] - 39s 46ms/step - loss: 0.0518 - accuracy: 0.9836 - precision: 0.9857 - recall: 0.9821 - val_loss: 0.0427 - val_accuracy: 0.9892 - val_precision: 0.9913 - val_recall: 0.9877
Epoch 3/30
844/844 [==============================] - 39s 46ms/step - loss: 0.0349 - accuracy: 0.9893 - precision: 0.9905 - recall: 0.9884 - val_loss: 0.0373 - val_accuracy: 0.9888 - val_precision: 0.9898 - val_recall: 0.9873
Epoch 4/30
844/844 [==============================] - 39s 46ms/step - loss: 0.0263 - accuracy: 0.9915 - precision: 0.9922 - recall: 0.9909 - val_loss: 0.0369 - val_accuracy: 0.9900 - val_precision: 0.9907 - val_recall: 0.9893
Epoch 5/30
844/844 [==============================] - 39s 46ms/step - loss: 0.0198 - accuracy: 0

In [ ]:
accuracy_class(model14)

Acurácia por Classe
Classe 0: 99.489796%
Classe 1: 99.911894%
Classe 2: 99.321705%
Classe 3: 99.702970%
Classe 4: 99.287169%
Classe 5: 98.991031%
Classe 6: 99.164927%
Classe 7: 99.319066%
Classe 8: 98.767967%
Classe 9: 98.414272%


In [ ]:
accuracy_total(model14)

Acurácia Total: 0.992500
Precision:      0.992797
Recall:         0.992400


Com a adição de mais uma camada de convolução o treinamento em relação a melhor versão anterior ficou mais lenta, porém conseguirmos consideravalmente aumentar todas as métricas! Sendo esse o melhor modelo até agora

## Sexta versão: CNN
Nessa versão utilizando o nosso melhor modelo até o momento, nós utilizamos o dropout, que é uma técnica de regularização para reduzir overfit e otimizar nossa rede, ele ignora aleatoriamente neurônios da camada indicada.

In [ ]:
model15 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.1),
        layers.Dense(250, activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(10, activation="softmax"),
    ]
)

model15.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()])

model15.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout (Dropout)            (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)              

In [ ]:
model15.fit(x_train, y_train, batch_size=64, epochs=30, validation_split=0.1)

Epoch 1/30
844/844 [==============================] - 40s 47ms/step - loss: 0.1868 - accuracy: 0.9426 - precision_1: 0.9666 - recall_1: 0.9228 - val_loss: 0.0665 - val_accuracy: 0.9808 - val_precision_1: 0.9833 - val_recall_1: 0.9792
Epoch 2/30
844/844 [==============================] - 40s 47ms/step - loss: 0.0586 - accuracy: 0.9813 - precision_1: 0.9837 - recall_1: 0.9792 - val_loss: 0.0410 - val_accuracy: 0.9880 - val_precision_1: 0.9896 - val_recall_1: 0.9872
Epoch 3/30
844/844 [==============================] - 40s 47ms/step - loss: 0.0405 - accuracy: 0.9874 - precision_1: 0.9887 - recall_1: 0.9861 - val_loss: 0.0375 - val_accuracy: 0.9892 - val_precision_1: 0.9910 - val_recall_1: 0.9887
Epoch 4/30
844/844 [==============================] - 40s 47ms/step - loss: 0.0321 - accuracy: 0.9895 - precision_1: 0.9906 - recall_1: 0.9888 - val_loss: 0.0333 - val_accuracy: 0.9912 - val_precision_1: 0.9915 - val_recall_1: 0.9902
Epoch 5/30
844/844 [==============================] - 40s 48ms/s

In [ ]:
accuracy_class(model15)

Acurácia por Classe
Classe 0: 99.387755%
Classe 1: 99.295154%
Classe 2: 99.224806%
Classe 3: 99.603960%
Classe 4: 99.185336%
Classe 5: 98.318386%
Classe 6: 99.060543%
Classe 7: 99.221790%
Classe 8: 99.794661%
Classe 9: 98.711596%


In [ ]:
accuracy_total(model15)

Acurácia Total: 0.991900
Precision:      0.992195
Recall:         0.991600


**Adicionar** a técnica de regularzição não foi o suficiente para ultrapassar os valores obtidos do modelo passado, porém ainda sim obtivemos taxas muito altas.

Achamos a Cnn um pouco mais intuitiva, então foi mais fácil de visualizar as diferenças de cada alteração ou ajuste que faziamos.